In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [ ]:
print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.1.0


In [ ]:
!ls wor

core.python3.48.1587222758  notebook.ipynb			    untitled
el_vectors.csv		    opa2vec_go_vectors.csv
MBdata_original.csv	    PAM50_feature_selection_with_MLP.ipynb


## 1000 genes selection

In [ ]:
path = "dl2vec_vectors.csv"
data = pd.read_csv(path)
labels = data.pop('Gene_ID')

print(data.shape)
print(labels.shape)

# Data in the format suitable for tensorboard projector
# with open('/content/drive/My Drive/MPhil/Data/embeddings_all.tsv', 'w') as f:
#   f.write(data.to_csv(sep='\t', header=False, index=False))
# with open('/content/drive/My Drive/MPhil/Data/embeddings_labels_all.tsv', 'w') as f:
#   f.write(labels.to_csv(sep='\t', header=False, index=False))

(17591, 200)
(17591,)


In [ ]:
path = "dl2vec_vectors.csv"
embeddings = pd.read_csv(path)
embeddings = embeddings.set_index('Gene_ID')

emb_dict={}
for index, row in embeddings.iterrows():
    emb_dict[index] = row.to_numpy()

emb_dict

{'FHIT': array([ 0.10939593, -0.06547377,  0.28014514, -0.08887822, -0.03980375,
         0.7390122 , -0.2732153 , -0.09070505,  0.20610735, -0.00857677,
         0.03203274, -0.1446024 ,  0.07092091,  0.2825429 ,  0.1662018 ,
         0.4559526 ,  0.11535546, -0.25788787,  0.01543026,  0.13928854,
         0.10330925, -0.12480167,  0.00471032,  0.14435346, -0.04712116,
        -0.02016609,  0.06904723,  0.34786493,  0.45686466,  0.20522194,
         0.34125835,  0.20209388,  0.02790004, -0.01656171, -0.22200875,
         0.06011262, -0.05207482, -0.120131  ,  0.09874071,  0.3276859 ,
        -0.47129694, -0.17223923,  0.15381879, -0.00375147,  0.24387652,
        -0.28866524, -0.2042454 ,  0.12981938, -0.04037623,  0.5034907 ,
         0.21494874,  0.6609837 , -0.08480049, -0.5475174 ,  0.11982924,
        -0.05970515,  0.34623984, -0.02816492,  0.24573912,  0.12405936,
        -0.05450421, -0.04720593, -0.05739425, -0.2289941 ,  0.00635485,
         0.39460388, -0.12295659, -0.184055

In [0]:
# from torch.utils.tensorboard import SummaryWriter

# vectors = data.to_numpy()
# metadata = labels.to_numpy()

# writer = SummaryWriter()
# writer.add_embedding(vectors, metadata)
# writer.close()

# %load_ext tensorboard
# %tensorboard --logdir=runs

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

df = pd.read_csv(path)
df = df.set_index('Gene_ID')

gene_name = "ESR1"
reference_emb = df.loc[gene_name].to_numpy().reshape(1, -1)
cos_distances = {}

for index, row in df.iterrows():
    similarity = cosine_distances(reference_emb, row.to_numpy().reshape(1, -1))
    cos_distances[index] = similarity
    # print(index, similarity)

cos_distances_sorted = {k: v for k, v in sorted(cos_distances.items(), key=lambda item: item[1])}

In [ ]:
cancer_drivers = ['FOXA1', 'BRCA2', 'TP53', 'PIK3CA', 'GATA3', 'ERBB2', 'WWP1', 'RARA']

for g in cancer_drivers:
  print("{} with position {} and cos distance {:1.3f}".format(g, list(cos_distances_sorted.keys()).index(g), cos_distances_sorted[g][0][0]))

FOXA1 with position 10 and cos distance 0.095
BRCA2 with position 1474 and cos distance 0.168
TP53 with position 8 and cos distance 0.095
PIK3CA with position 72 and cos distance 0.116
GATA3 with position 635 and cos distance 0.150
ERBB2 with position 97 and cos distance 0.119
WWP1 with position 6588 and cos distance 0.212
RARA with position 625 and cos distance 0.149


# Dataset

In [ ]:
!ls /datasets/mbdata

MBdata_all.csv


In [ ]:
path = "/datasets/mbdata/MBdata_all.csv"
data = pd.read_csv(path)

/opt/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (15,16,18,20,21,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data = data[data.Pam50Subtype != '?']

In [ ]:
data

,Unnamed: 0,METABRIC_ID,Cohort,Age_At_Diagnosis,Breast_Tumour_Laterality,Date_Of_Diagnosis,Last_Followup_Status,NPI,ER_Status,Inferred_Menopausal_State,...,CNA_FOXM1,CNA_EXOC3,CNA_ARMC7,CNA_KCTD2,CNA_PDCD10,CNA_RHOD,CNA_CFL1,CNA_YBX1,CNA_TRIM24,CNA_RAB11FIP3
0,0,MB-0002,1,43.19,r,2000/12,a,4.020,pos,pre,...,0,0,-1,-1,0,2,2,0,-1,0
1,1,MB-0005,1,48.87,r,2001/02,d-d.s.,4.030,pos,pre,...,0,0,0,0,0,1,1,0,2,1
2,2,MB-0006,1,47.68,r,2001/02,a,4.050,pos,pre,...,0,0,0,0,0,0,0,0,1,0
3,3,MB-0010,1,78.77,l,2001/03,d-d.s.,4.062,pos,post,...,0,0,0,0,0,1,1,0,0,2
4,4,MB-0014,1,56.45,r,2001/04,a,4.020,pos,post,...,0,-1,0,0,0,0,0,0,0,0
5,5,MB-0020,1,70.00,l,2001/03,d-d.s.,6.130,neg,post,...,0,0,0,0,0,0,0,0,0,0
6,6,MB-0022,1,89.08,l,2001/04,d-o.c.,4.058,pos,post,...,0,0,0,0,0,1,1,-1,0,1
7,7,MB-0667,1,76.24,l,2001/03,d-d.s.,6.068,pos,post,...,0,0,0,0,0,0,0,0,0,0
8,8,MB-0028,1,86.41,r,2001/04,d,5.032,pos,post,...,-1,0,1,1,0,0,0,-1,-1,0
9,9,MB-0035,1,84.22,l,2001/04,d-d.s.,3.056,neg,post,...,-1,0,-1,-1,0,0,0,0,1,0


In [ ]:

features = data.filter(regex='^GE.*')
features = features.astype('float64')
print(features)

target = data.pop('Pam50Subtype')
pam50_lables = {
    'Normal': 0,
    'LumA': 1,
    'LumB': 2,
    'Basal': 3,
    'Her2': 4
}
target = target.apply(lambda x: pam50_lables[x])

       GE_RERE  GE_RNF165  GE_CD049690  GE_BC033982   GE_PHF7   GE_CIDEA  \
0     9.013876   5.748717     5.699735     5.419119  5.611212   6.199492   
1     7.963493   5.553056     5.472386     5.265556  5.793398   6.489781   
2     8.177157   5.391160     5.529795     5.011812  5.807604   5.319779   
3     8.385715   5.453305     5.509027     5.225599  6.061952   5.596764   
4     8.653761   5.959891     5.446798     5.529548  5.915674   8.507177   
...        ...        ...          ...          ...       ...        ...   
1975  8.429599   6.082983     5.354810     5.325897  5.562396   7.079116   
1976  8.850144   6.440723     5.210231     5.419619  5.828949   5.997769   
1977  9.289050   6.882522     5.560592     5.233888  5.608357  10.720036   
1978  9.733945   6.335700     5.226621     5.159931  5.516629   7.083591   
1979  8.275924   6.919406     5.395820     5.458220  5.807318   5.572809   

      GE_PAPD4  GE_AI082173  GE_SLC17A3    GE_SDS  ...  GE_BX115874  \
0     8.341091  

In [ ]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if max_value != min_value:
          result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
# features = data.filter(top_1000)
features = data.filter(regex='^GE.*')
features = normalize(features)
features.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

## Experiments

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn import metrics
from sklearn.model_selection import cross_validate, KFold, train_test_split
import random


In [ ]:
top_opa2vec = ['GE_'+g for g,v in cos_distances_sorted.items() if 'GE_'+g in list(data.columns)]


In [ ]:
features_org = features.copy()

In [ ]:
features = features.filter(top_opa2vec[:1000])

x_train = features.values[:]
y_train = target.values[:]

acc = []
kf = KFold(n_splits=5)

for train, valid in kf.split(x_train, y_train):
  y_train = target.values[:]
  b = np.zeros((y_train.size, 5))
  b[np.arange(y_train.size),y_train] = 1
  y_train = b

  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),# use_bias=False),
    #tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(5, activation='softmax')
  ])

  adam = tf.keras.optimizers.Adam(learning_rate=0.00005)
  model.compile(optimizer=adam,
                loss='categorical_crossentropy',
                metrics=['accuracy'])


  model.fit(x_train[train], y_train[train], epochs=50,  verbose=0)
  model.evaluate(x_train[valid], y_train[valid], verbose=2)
  y_pred=model.predict(x_train[valid])
  y_pred=np.argmax(y_pred, axis=1)
  score = metrics.accuracy_score(np.argmax(y_train[valid], axis=1), y_pred)
  acc.append(score)

print("---- Mean acc:", np.mean(acc))

395/395 - 0s - loss: 0.5041 - accuracy: 0.8025
395/395 - 0s - loss: 0.4439 - accuracy: 0.8228
395/395 - 0s - loss: 0.4537 - accuracy: 0.8278
395/395 - 0s - loss: 0.4978 - accuracy: 0.7899
394/394 - 0s - loss: 0.5962 - accuracy: 0.7690
---- Mean acc: 0.8024147015356935


In [ ]:
top_opa2vec = ['GE_'+g for g,v in cos_distances_sorted.items() if 'GE_'+g in list(data.columns)]
print(top_opa2vec[:10])

results = {
    1000: [],
    100: [],
    10: [],
    5: [],
    2: [],
    1: []
}

features = data.filter(regex='^GE.*').copy()
features = normalize(features)
features.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

for n in [1000,100,10,5,2,1]:
  for _ in range(10):
    features = features.filter(top_opa2vec[1:n+1])
    features_size = len(list(features.columns))

    names = list(features.columns)
    rf = RandomForestClassifier()
    kf = KFold(n_splits=5)
    feature_imp = []
    acc = []
    acc_test = []
    pred = []
    ref = []

    x_train = features.values[:]
    y_train = target.values[:]

    for train, valid in kf.split(x_train, y_train):
      b = np.zeros((y_train.size, y_train.max()+1))
      b[np.arange(y_train.size),y_train] = 1
      y_train = b

      model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(32, activation='relu'),
        # tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(5, activation='softmax')
      ])

      model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    

      model.fit(x_train[train], y_train[train], epochs=50)
      model.evaluate(x_train[valid], y_train[valid], verbose=2)
      y_pred=model.predict(x_train[valid])
      acc.append(metrics.accuracy_score(y_train[valid], y_pred))
      pred += list(y_pred)
      ref += list(y_train[valid])

    acc = np.mean(acc)
    results[n].append(acc*100)
    print(f"Accuracy:{acc:.3}, Feature set size:{features_size}")

print(results)
for f, acc in results.items():
  print(f"{np.mean(acc):2.2f}% +- {np.std(acc):.2}%")

['GE_ESR1', 'GE_PARK7', 'GE_MYC', 'GE_BAX', 'GE_CEBPB', 'GE_HIF1A', 'GE_STK11', 'GE_HMGB1', 'GE_TP53', 'GE_TFF1']
Train on 1579 samples
Epoch 1/50
  32/1579 [..............................] - ETA: 4s

ValueError: Can not squeeze dim[1], expected a dimension of 1, got 5 for 'metrics/accuracy/Squeeze' (op: 'Squeeze') with input shapes: [?,5].

In [0]:
for f, acc in results.items():
  print(f"{np.mean(acc):2.2f}% +- {np.std(acc):.2}")

In [0]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

matrix = confusion_matrix(top_features_test, top_features_pred)
print(matrix)


plot_labels=pam50_lables.keys()

figure = plt.figure(figsize=(2, 2))
sns.heatmap(matrix, annot=True, fmt="d", cmap=plt.cm.Blues, xticklabels=plot_labels , yticklabels=plot_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()